In [132]:
from PIL import Image
import numpy as np
from collections import Counter
import colorsys

# Extended color-to-note mapping with semitones
COLOR_TO_NOTE = {
    'C': (255, 0, 0),          # Red
    'C#': (255, 102, 102),     # Light Red
    'D': (255, 165, 0),        # Orange
    'D#': (255, 140, 0),       # Dark Orange
    'E': (255, 255, 0),        # Yellow
    'F': (0, 255, 0),          # Green
    'F#': (102, 255, 102),     # Light Green
    'G': (0, 0, 255),          # Blue
    'G#': (102, 178, 255),     # Light Blue
    'A': (75, 0, 130),         # Indigo
    'A#': (238, 130, 238),     # Violet
    'B': (186, 85, 211)        # Light Purple
}

def calculate_average_brightness(image):
    """
    Calculate the average brightness of an image.
    Converts the image to grayscale and computes the mean pixel value.
    """
    grayscale_image = image.convert('L')  # Convert to grayscale
    pixels = np.array(grayscale_image)
    average_brightness = np.mean(pixels)
    return average_brightness


def resize_image(image, max_size=500):
    """Reduz o tamanho da imagem para melhorar a performance."""
    image.thumbnail((max_size, max_size), Image.BILINEAR)
    return image


def calculate_saturation(image):
    """
    Calculate the average saturation of an image.
    Converts the image to RGB, then to HSV to extract saturation values.
    """
    rgb_image = image.convert('RGB')  # Ensure image is in RGB
    pixels = np.array(rgb_image)
    # Reshape to a list of pixels
    reshaped_pixels = pixels.reshape(-1, 3)
    # Convert RGB to HSV and extract saturation
    hsv_pixels = [colorsys.rgb_to_hsv(r/255.0, g/255.0, b/255.0) for r, g, b in reshaped_pixels]
    saturations = [s for h, s, v in hsv_pixels]
    average_saturation = np.mean(saturations)
    return average_saturation

def find_nearest_note_color(pixel, color_map):
    """
    Find the nearest note color for a given pixel based on Euclidean distance.
    """
    min_distance = float('inf')
    nearest_note = None
    for note, color in color_map.items():
        distance = np.linalg.norm(np.array(pixel) - np.array(color))  # Euclidean distance
        if distance < min_distance:
            min_distance = distance
            nearest_note = note
    return nearest_note

def find_rainbow_colors(image, color_map):
    """
    Find and count the frequency of each rainbow color (including semitones) in the image.
    """
    rgb_image = image.convert('RGB')  # Ensure image is in RGB
    pixels = np.array(rgb_image)
    # Reshape to a list of pixels
    reshaped_pixels = pixels.reshape(-1, 3)
    pixel_counter = Counter(map(tuple, reshaped_pixels))

    # Initialize frequency dictionary
    color_frequency = {note: 0 for note in color_map.keys()}

    for pixel, count in pixel_counter.items():
        note = find_nearest_note_color(pixel, color_map)
        if note:
            color_frequency[note] += count

    # Order colors by frequency
    ordered_colors = sorted(color_frequency.items(), key=lambda x: x[1], reverse=True)
    return ordered_colors

def assign_music_features(brightness, ordered_colors, saturation):
    """
    Assign pitch, tone sequence, and tempo based on image features.
    """
    # Assign pitch based on brightness (A4 = 440Hz is middle range)
    base_pitch = 440  # Middle A pitch
    pitch = int(base_pitch * (brightness / 255))  # Scale brightness to pitch range

    # Assign sequence of tones based on color order (already mapped to notes)
    tone_sequence = [note for note, freq in ordered_colors[:4] if freq > 0]

    # Assign tempo based on saturation (higher saturation, faster tempo)
    base_tempo = 240  # Base tempo in BPM
    tempo = int(base_tempo * saturation)  # Scale saturation to tempo
    if tempo > 160:
        tempo = 160

    return pitch, tone_sequence, tempo

def process_image(image_path, resize = True):
    """
    Process the image to extract features and assign musical attributes.
    """
    try:
        image = Image.open(image_path)
    except IOError:
        print(f"Error: Unable to open image at path '{image_path}'. Please check the file path.")
        return
    
    if resize == True:
        image = resize_image(image)

    # Extract features
    brightness = calculate_average_brightness(image)
    saturation = calculate_saturation(image)
    ordered_colors = find_rainbow_colors(image, COLOR_TO_NOTE)

    # Assign music features
    pitch, tone_sequence, tempo = assign_music_features(brightness, ordered_colors, saturation)

    # Print the results
    print(f"Average Brightness: {brightness:.2f}")
    print("Color Frequencies (Ordered):")
    for color, freq in ordered_colors:
        print(f"  {color}: {freq}")
    print(f"Average Saturation: {saturation:.2f}")
    print(f"Assigned Pitch (Hz): {pitch}")
    print(f"Tone Sequence: {' - '.join(tone_sequence)}")
    print(f"Assigned Tempo (BPM): {tempo}")

    return {
        "brightness": brightness,
        "ordered_colors": ordered_colors,
        "saturation": saturation,
        "pitch": pitch,
        "tone_sequence": tone_sequence,
        "tempo": tempo
    }

if __name__ == "__main__":
    # Example usage
    image_path = 'james_webb_images/heic0004_large.jpg'  # Replace with your image path
    features = process_image(image_path)

Average Brightness: 5.98
Color Frequencies (Ordered):
  A: 246569
  B: 1051
  G#: 913
  F#: 665
  A#: 517
  F: 172
  C#: 112
  D#: 1
  C: 0
  D: 0
  E: 0
  G: 0
Average Saturation: 0.92
Assigned Pitch (Hz): 10
Tone Sequence: A - B - G# - F#
Assigned Tempo (BPM): 160


In [133]:
music_features = [features[feature] for feature in ['tone_sequence', 'tempo']]

In [139]:
import requests
import json
import os

api_key = 'sk-FDti6mHcD9tIT6bNgmrHvPd9bUcUqCJRfJLhynB9OkT3BlbkFJSvdRzIA5BrDL_GKIhBkFTKl-fZKmmSYoUMizf8ZEMA'
input_text = 'In the cosmic expanse, a nebula blooms, Swirls of gas and dust like a dream in hues. Golden cliffs rise, sculpted by stellar winds, Reaching into the void where eternity begins.'

tone_sequence = music_features[0]
tempo = music_features[1]

class SongMaker:
    def __init__(self, api_key, input_text):
        self.api_key = api_key
        self.input_text = input_text
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }


    def make_song(self):
        """Makes an ABC format song based on a input text."""
        payload = {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "user",
                    "content": "RolePlay as a musical bot. Generate a a music in ABC format based on the input text."
                },
                {
                    "role": "user",
                    "content": f"Style: Symphonic, in the style of Hans Zimmer. Aim for a longer sequence, of about a minute. Tones should include: {tone_sequence}. Tempo: {tempo} BPM. Pitch: Low-pitch. Aim for a very inspirational song, that motivates exploration. It should be a calm song."
                },
                {
                    "role":"user"
                    "content":f"Base the song on the following poem: {self.input_text}"
                }
                {
                    "role": "user",
                    "content": "Make a melodic synphony using the piano."
                },
                {
                    "role": "user",
                    "content": "Remove any introductions or explanations. Send only the requested text."
                },
            ],
            "max_tokens": 600
        }

        response = requests.post("https://api.openai.com/v1/chat/completions",
                                 headers=self.headers, json=payload)
        return response.json()


generate_song = SongMaker(api_key, input_text)

# Make a song
song = generate_song.make_song()
print(song)

{'id': 'chatcmpl-AFPqXu2CabH7EeZYKCyBuOq8zAIU6', 'object': 'chat.completion', 'created': 1728237637, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```\nX:1\nT:Cosmic Bloom\nM:4/4\nL:1/8\nQ:1/4=160\nK:A\nV:1 clef=treble\n|: A2 B2 G2 G2 | F#4 A4 | A2 B2 G2 G2 | F#4 A4 |\n|  A2 B2 G2 G2 | F#4 A4 | B2 A2 F#2 G2 | F#4 A4 |\n|  B2 A2 G2 A2 | F#4 A4 | [G2B2] [G2B2] [G2B2] [G2B2] | [F#2A2] [F#2A2] [F#2A2] [F#2A2] |\n|  B2 A2 F#2 G2 | F#4 A4 | A2 B2 G2 G2 | F#4 A4 :|\n```\n', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 174, 'completion_tokens': 216, 'total_tokens': 390, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'system_fingerprint': 'fp_f85bea6784'}


In [142]:
print(song['choices'][0]['message']['content'])

```
X:1
T:Cosmic Bloom
M:4/4
L:1/8
Q:1/4=160
K:A
V:1 clef=treble
|: A2 B2 G2 G2 | F#4 A4 | A2 B2 G2 G2 | F#4 A4 |
|  A2 B2 G2 G2 | F#4 A4 | B2 A2 F#2 G2 | F#4 A4 |
|  B2 A2 G2 A2 | F#4 A4 | [G2B2] [G2B2] [G2B2] [G2B2] | [F#2A2] [F#2A2] [F#2A2] [F#2A2] |
|  B2 A2 F#2 G2 | F#4 A4 | A2 B2 G2 G2 | F#4 A4 :|
```



In [143]:
# Let's attempt to convert the revised ABC notation to MIDI using a different approach

# Define the revised ABC notation
abc_code_final = """
X:1
T:Cosmic Bloom
M:4/4
L:1/8
Q:1/4=160
K:A
V:1 clef=treble
|: A2 B2 G2 G2 | F#4 A4 | A2 B2 G2 G2 | F#4 A4 |
|  A2 B2 G2 G2 | F#4 A4 | B2 A2 F#2 G2 | F#4 A4 |
|  B2 A2 G2 A2 | F#4 A4 | [G2B2] [G2B2] [G2B2] [G2B2] | [F#2A2] [F#2A2] [F#2A2] [F#2A2] |
|  B2 A2 F#2 G2 | F#4 A4 | A2 B2 G2 G2 | F#4 A4 :|
"""

# Attempt to convert the ABC notation to MIDI using music21
try:
    from music21 import converter, midi

    # Convert ABC to music21 stream
    abc_work_final = converter.parse(abc_code_final, format='abc')

    # Create a MIDI file from the music21 stream
    midi_fp_final = 'cosmic_bloom_intro_final.mid'
    mf_final = midi.translate.music21ObjectToMidiFile(abc_work_final)
    mf_final.open(midi_fp_final, 'wb')
    mf_final.write()
    mf_final.close()

    midi_fp_final
    print("done.")

except Exception as e:
    print(str(e))

done.


In [144]:
pip install mido pyfluidsynth pydub numpy scipy

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
  Created wheel for pyfluidsynth: filename=pyFluidSynth-1.3.3-py3-none-any.whl size=19203 sha256=96a32cc22944b3a1d4093f6f6dfad4f6fe6fda21997da5e977910431e9528511
  Stored in directory: /home/gustamatos/.cache/pip/wheels/e6/9c/2b/b4b194cbb100d6f20136a22b2f76cd329b1a21139ed26c9775
Successfully built pyfluidsynth

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
